## Name Binding (What You Think of as Assignment)
Whenever you use assignment (=), you can think about it as putting a name tag (named with the variable name) onto the data--this is called name binding. Yoy can use the `id()` function to see the object's memory address. Hence, for the duration of the object's existence (ie not deleted), then the id will be unique and unchanged. Python's assigment (=) NEVER copies data--it merely puts another name tag on the same object. The `is` keyword in Python is highly related to `id()` in that `is` detects whether the identity of 2 objects are the same--if the 2 variables are pointing to same memory address. If `x is y` is True, then `id(x) == id(y)`.

If you want to see how many name tags are attached to the same object, then use `sys.getrefcout`. Python internally has a reference count for every object--once the reference count goes to 0, then the object is deleted from memory.  
Notice this caveat from the documentation:  
```The count returned is generally one higher than you might expect, because it includes the (temporary) reference as an argument to getrefcount().```  
Deleting a variable will remove the name tag. Python's `del` statement doesn't directly delete a variable. Instead `del` deletes the name binding (the name tag) onto the object. Once an object's reference count goes to 0, then Python's garbage collector will finally remove the object from memory.

Python has data structures that are mutable (list, set, dictionary) and some are immutable (tuple, string, numeric [float, int, bool]). 

In [1]:
import sys

a = []
print("reference count: {}; memory location: {}".format(sys.getrefcount(a), id(a)))

reference count: 2; memory location: 2039532539976


In [2]:
# a list is a mutable object
a = []
print(sys.getrefcount(a))
b = a
print(sys.getrefcount(a))
print(a is b)
print(id(a), id(b))
a.append(None)
print(a, b)

2
3
True
2039532511112 2039532511112
[None] [None]


In [3]:
# an tuple is immutable--you cannot make a number into another number. That will occupy different memory locations
a = (1, )
print(sys.getrefcount(a))
b = a
print(sys.getrefcount(a))
print(a is b)
print(id(a), id(b))
a += (2,)
print(a, b)
print(a is b)
print(id(a), id(b)) # notice that id(a) has changed memory location
print(sys.getrefcount(a)) # notice that the reference count has fell back to 2

2
3
True
2039533079408 2039533079408
(1, 2) (1,)
False
2039515341704 2039533079408
2


R is pass by value--that means when you pass a dataframe into a function, the dataframe is copied. If you mutate the dataframe inside an R function, nothing will happen to the original dataframe.  
This is not true in Python.  In contrast, Python's assigment (=) NEVER copies data, not even in a function's argument assignment. Hence, when you pass a variable into the function, that variable can be mutated if it is a mutable type. Python is not pass by value nor pass by reference. Instead Python is pass by object. 

In [4]:
# mutable object changed
def appender(lst):
    lst.append(None)
    
a = []
print(a)
appender(a)
print(a)

[]
[None]


In [5]:
# immutable object remains unchanged
def concatenate(string):
    string = "Hello " + string # this new `string` variable is no longer bound to the same object. 
    print(id(string), string) # You have moved the "name tag" to a different object.

string = "ML Study Group"
print(id(string), string)
concatenate(string)
print(id(string), string)

2039533281712 ML Study Group
2039533253304 Hello ML Study Group
2039533281712 ML Study Group


If fact, you change to multiple assignment. The right-most expression is evaluated first and then bound to the variables left to right.

In [6]:
a = b = 9000 # bind 900 to `a` first, then `b`
print(a, b)
print(a is b)

9000 9000
True


In [7]:
a, b = 1, 2 # tuple unpacking
a, b = b, a # this is a safe operation in Python. You don't need a temporary third variable.
print(a, b)

2 1


Here's a common ~~n00b~~ beginner mistake. There's nothing special about built-in types: you can overwrite it. And here's how you undo it.

In [8]:
list = list(range(10)) # I just lost the ability to create lists
list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
# how do I get tuple() function back?
list, my_tuple = type(list), list

print(my_tuple)
print(list(range(5)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 4]


In [10]:
# If you combine tuple unpacking and multiple assignment, you get this is fun puzzle. 
# The values for a and b are swapped and then swapped back. Basically, nothing happens.
a, b = 1, 2

(b, a) = (a, b) = (a, b)
a, b

(1, 2)

To prove that multiple assigment happens left to right, so I use a `property` attribute. I'll explain `property` later.

In [11]:
class SillyAssignment:
    def __init__(self, value):
        self.id = value
        self._gotit = None
        self._counter = 0
    
    @property
    def gotit(self):
        print("Getter {}: {}".format(self.id, self._gotit))
        return self._gotit
        
    @gotit.setter
    def gotit(self, value):
        self._counter += 1
        print("Setter {} to {}; set {} time(s)".format(self.id, value, self._counter))
        self._gotit = value

In [12]:
a, b = 1, 2

s1 = SillyAssignment("s1")
s2 = SillyAssignment("s2")

In [13]:
s2.gotit, s1.gotit = s1.gotit, s2.gotit = (a, b)

Setter s2 to 1; set 1 time(s)
Setter s1 to 2; set 1 time(s)
Setter s1 to 1; set 2 time(s)
Setter s2 to 2; set 2 time(s)


Magic trick: Any time you see a magic trick, it's just for fun. Don't think about it too hard--unless you like to ;)  
A recursive list. Instead of a recursive function, perhaps this is recursive analog of a data structure.

In [14]:
a = a[0] = [None]

print(a)
a is a[0] is a[0][0]
print(id(a), id(a[0]), id(a[0][0]))

[[...]]
2039533309256 2039533309256 2039533309256


Another magic tricK: twisting 2 lists together like a Mobius strip

In [15]:
a, b = b[0], a[0] =  [None], [None]

print(a, b)
print(a[0] is b, b[0] is a)
print(a is a[0][0])
print(id(a), id(a[0][0]))

[[[...]]] [[[...]]]
True True
True
2039533284360 2039533284360


## Multiple Comparison
Comparisons can also be chained together. The motivation is some function calls can take a long time, so you don't want to call it twice. The chaining is equivalent to `left_expression and right_expression`.

In [1]:
print(1 < 2 < 3) # equivalent to (1 < 2) and (2 < 3)
print(1 < 3 > 2) # equivalent to (1 < 3) and (3 > 2)

True
True


In [2]:
import time

def slow_call(value, nap_time):
    time.sleep(nap_time)
    return value

In [3]:
%%time
(1 < slow_call(2, 5)) and (slow_call(2, 5) < 3)

Wall time: 10 s


True

In [4]:
%%time
1 < slow_call(2, 5) < 3 # takes half the time!

Wall time: 5 s


True

## Short Circuiting: and + or
`or` and `and` have a cool functionality in that do not have to evaluate all expressions. If `or` has a first argument that is logically `True`, then it will not evaluate the second argument. Short circuiting is similar to lazy evaluation in that not all expressions are executed.

In [5]:
print(5 or False)
print(5 or print("HI")) # print() returns None
print(5 or 6 or 7 or 8)

5
5
5


`and` is the opposite in that it evaluates expressions left to right until it sees the first logically `False` argument and returns it. If there are no falsy (real word!) expressions, then return the last argument (which necessarily is truthy).

In [6]:
print(1 and 2 and 3 and 0 and 4 and 5)
print(1 and 2 and 3 and 4 and 5) # prints 5
print(False and 0 and "")
print(0 and "" and False)

0
5
False
0


The value comes in handy if you are evaluating multiple conditions. Depending on whether you have `or` or `and`, then you want to put the first expression you think will short circuit the rest of the condition quicker.

In [7]:
import time

def slow_call(value, nap_time):
    time.sleep(nap_time)
    return value

In [8]:
%%time
if slow_call(True, 1) and slow_call(True, 2): # has the wait the full 3 seconds
    print("HI") # executed

HI
Wall time: 3.01 s


In [9]:
%%time
if slow_call(True, 1) or slow_call(True, 2): # since the first expression is truthy, then skip the second one
    print("HI") # executed

HI
Wall time: 1 s


In [10]:
%%time
if slow_call(False, 1) or slow_call(False, 2): # has the wait the full 3 seconds
    print("HI") # not executed

Wall time: 3 s


In [11]:
%%time
if slow_call(False, 1) and slow_call(False, 2): # since first expression is falsy, then skip the second one
    print("HI") # not executed

Wall time: 1 s


In [12]:
%%time
if slow_call(True, 1) or slow_call(True, 2) or slow_call(True, 3) or slow_call(True, 4): # still only takes 1 second
    print("HI")

HI
Wall time: 1 s


## What's My Name? Variable Scoping
When you use multiple variables with the same name, how do you know which variable you get?  
`Namespaces` : A namespace is a container where names are mapped to objects. Hence, you have same variable names associated to different objects in different spaces.  
`Scope`: A scope defines the hierarchical order in which the namespaces have to be searched in order to obtain the mappings of name to object. Python follows the LEGB rule: `Local, Enclosing, Global, Built-in`. If a variable is not found in the local namespace, go to the outer namespaces. If not found at all, then raise `NameError` exception. Variables with the same name in the same namespace will overwrite each other--only 1 can exist at a time.

The vocabulary words `namespace` and `scope` isn't very important. What's more important is understanding how they work.

<p align="center"><img src="images/LEGB_Variable_Scoping.png"></p>
`Local`: inside a function, look to see if variable is defined. If it is there, use that one.

In [1]:
a = 1 # this is a global variable
def func():
    a = 2 # this is the local variable
    return a
print(func())
print(a)

2
1


`Enclosing`: if not local variable found inside nested function, look for the variable in the outer function.

In [2]:
a = 1 # this a global variable
def func1():
    a = 2 # get this enclosing variable
    def func2():
        return a
    return func2()
print(func1())
print(a)

2
1


In [3]:
a = 1 # this a global variable
def func1():
    a = 2 # this is still an enclosing variable as it is not global
    def func2():
        def func3():
            return a
        return func3()
    return func2()
print(func1())
print(a)

2
1


`Global`: variables defined at the module level. In your function call, if variables are not defined locally or enclosing, go to the module level to find the variable.

In [4]:
a = 1 # global variable
def func1():
    def func2():
        return a
    return func2()
print(func1())
print(a)

1
1


In [5]:
# recursion uses scoping to complete itself
def fibonacci(n):
    if (n == 1) or (n == 2):
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2) # where is fibonacci() defined? It's a global variable

fibonacci(10)

55

In [6]:
# Magic trick: you can "twist" functions to see how LEGB really works
def func1(n):
    def func2(n):
        if n:
            print("I'm inside func2. The number is {}".format(n))
            return func1(n - 1)
        return

    if n:
        print("I'm inside func1. The number is {}".format(n))
        return func2(n - 1)
        
print(func1(10))

I'm inside func1. The number is 10
I'm inside func2. The number is 9
I'm inside func1. The number is 8
I'm inside func2. The number is 7
I'm inside func1. The number is 6
I'm inside func2. The number is 5
I'm inside func1. The number is 4
I'm inside func2. The number is 3
I'm inside func1. The number is 2
I'm inside func2. The number is 1
None


`Global` variables are not _truly_ global in the sense that variables cannot communicate between modules. If `pandas` and `sklearn` both have a variable `a`, then they cannot communicate with each other. `pandas.a` does not know about `sklearn.a`.

In [7]:
%%writefile temp1.py
a = 1

Overwriting temp1.py


In [8]:
%%writefile temp2.py
a = 2

Overwriting temp2.py


In [9]:
%%writefile temp3.py
print(a)

Overwriting temp3.py


In [10]:
import temp1
print(temp1.a)

1


In [11]:
import temp2
print(temp2.a)

2


In [12]:
import temp3 # fails to load

NameError: name 'a' is not defined

In [13]:
a = 1 # even if you define it now, temp3 cannot be imported since `a` in temp3 does not exist
import temp3

NameError: name 'a' is not defined

`Built-ins` are the things that you generally use without thinking: `sum()`, `range()`, `list`. These variables/functions are like global globals in the sense that they exist between modules.

In [14]:
def printer():
    print("Money tree: sorry, my gold is green--the color of envy!") # where does the function print() come from? It's a built-in!

Magic trick: how do you get global globals? How can you communicate between different modules? You can do that by hacking away at `builtins`. Built-in functions/variables are not special--they can be overwritten like everything else. As you can see, you can cause some chaos.

In [15]:
%%writefile temp1.py
import builtins

builtins.len = lambda _: "len() is now gone"
builtins.list = tuple
builtins.blah_blah_blah = "Ke$ha"


Overwriting temp1.py


In [16]:
%%writefile temp2.py
import temp1

print(len("HI")) # len no longer works
print(list(range(10))) # became a tuple
print(blah_blah_blah) # some random variable now exists

Overwriting temp2.py


In [17]:
!python temp2.py

len() is now gone
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
Ke$ha


If there are 2 variables of the same name in the same namespace, then there can only be one! The reason is that namespaces are themselves implemented with a dictionary. Hence, the most recent one overwrites the previous one.

In [18]:
a = 1 # global variable will be overwritten by the following function
def a(): # this function overwrites/replaces the previous assignment. The value of 1 is totally lost/gone.
    def b():
        return a # refers to the outer function and thus becomes a recursive call
    return b()
print(a())
a() is a()() is a()()()

<function a at 0x0000022C97C47400>


True

## (Re)assignment within inner namespaces: Use with Caution
As a general rule, variables in inner namespaces cannot assign to outer namespaces. However, you can mutative variables in inner namespaces; for example lists are mutatable. There is an exception to this rule by using `global` or `nonlocal` (which are the only declarative statements that Python has). `global` tells Python you are using a global variable. `nonlocal` is to assign/reassign for enclosing variables. I have never used `global` or `nonlocal` in real code, so use them sparingly, as they violate the Principle of Least Surprise.

In [19]:
a = []
def func():
    a.append(None)
print(a)
func()
print(a) # a is mutated

[]
[None]


In [20]:
a = [] # global
def inner():
    a = [] # local
    a.append(None) # refers to the local variable
print(a)
inner()
print(a) # a is not mutated or assigned. Basically nothing has happened

[]
[]


In [21]:
a = 1
def reassign():
    global a
    a = 2

print(a)
reassign()
print(a)

1
2


In [22]:
a = 1 # re-assign this variable
def reassign():
    a = 2
    def func():
        global a
        print(a, "I am inside nested function")
        a = 3
    func()
    print(a, "I am in the enclosing variable") # nothing changed here
print(a, "before global reassignment")
reassign()
print(a, "after global reassignment")

1 before global reassignment
1 I am inside nested function
2 I am in the enclosing variable
3 after global reassignment


In [23]:
def global_assign():
    global xyz
    xyz = 1
    
try:
    del xyz
    print("deleted xyz")
except NameError:
    print("xyz doesn't exist")

try:
    print(xyz)
except NameError:
    print("xyz doesn't exist")

global_assign()
print(xyz)

xyz doesn't exist
xyz doesn't exist
1


In [24]:
a = 1
def func1():
    a = 2 # reassign this enclosing variable
    def func2():
        nonlocal a
        a = 3
    print(a, "enclosing variable before reassignment")
    func2()
    print(a, "enclosing variable after reassignment")
print(a, "global variable unchanged")
func1()

1 global variable unchanged
2 enclosing variable before reassignment
3 enclosing variable after reassignment


## Executive Power: `exec` and `eval`
These 2 functions are very powerful. As Uncle Ben once said: with great power comes great responsibility. In practice, you will rarely have to use these.

`exec()` takes a string and will execute the string as if it were Python code. You should never give untrusted sources to run arbitrary code, since they can delete your entire hard drive. `exec()` is a function in Python 3 (that always returns `None`) whereas `exec` is a statement in Python 2 (which cannot be saved). This is the same upgrade for `print()` which is a statement in Python 2 and a function in 3.

In [1]:
exec("print('HI')")

exec("import sys") # can mutate the environment
print(sys.version) # sys is now imported

lister = []
exec("lister.append(42)") # can mutate the environment
print(lister)

exec("x = 1; x += 2; print(x)") # can have multiple statements

print(exec("")) # exec returns None

exec("print({} + {})".format(1, 2)) # you can dynamically generate a string

In [2]:
print(print("HI")) # the inner print() returns None. This syntax cannot be used in Python 2

HI
None


`eval()` is bit safer: you are allowed 1 expression, no statement. If you put an expression, you can save that value.

In [3]:
x = eval("1 + 2")
print(x)

3


In [4]:
print(eval("(1, 2) + (3, 4)")) # no statement, the expression must return something

(1, 2, 3, 4)


In [5]:
eval("1 + 2; 3 + 4") # only 1 expression

SyntaxError: invalid syntax (<string>, line 1)

In [6]:
eval("x = 10") # no statement, the expression must return something

SyntaxError: invalid syntax (<string>, line 1)

There's an even safer option: `ast.literal_eval`, which on evaluates 1 literal object. It cannot understand variables.

In [7]:
import ast

print(ast.literal_eval("1"), ast.literal_eval("None"), ast.literal_eval("()"), ast.literal_eval("True"))

1 None () True


In [8]:
print(eval("[] + []"))
print(ast.literal_eval("[] + []")) # not 1 element

[]


ValueError: malformed node or string: <_ast.BinOp object at 0x000001B07865D208>

In [9]:
x = 1
print(eval("x"))
print(ast.literal_eval("x")) # not a literal object, is a variable

1


ValueError: malformed node or string: <_ast.Name object at 0x000001B07865D9B0>

### Style Guide

In [1]:
# I usually use this type of syntax for prints where I put all the variables at the end. This is up to your personal preference
first_name = 'Peter'
last_name = 'Pan'
print("Hello, my name is {} {}".format(first_name, last_name))
# You can change the order using named arguments and also repeat the arguments
print("Last name: {last_name}; first name: {first_name}. Again my name is {first_name} {last_name}".format(first_name=first_name, last_name=last_name))

# f**** it! You can use the new f-string syntax that came out in Python 3.6
print(f"Hello, my name is {first_name} {last_name}")
# have you ever considered nested string formatting? ;-)
print(f"{f'I am {42 -29} years old!'}")

Hello, my name is Peter Pan
Last name: Pan; first name: Peter. Again my name is Peter Pan
Hello, my name is Peter Pan
I am 13 years old!


In [ ]:
# Context manager
# If you like, use this syntax since it guarantees that the file will be closed after everything in the code block is run.
# It just saves one line (don't have to close file manually) and also make the code block for what you want to do
# the file very obvious due to the indentation.
with open('file_here') as f:
    f.read()

In [2]:
# Instead of \ to denote line continuation, I use parenthesis--also works for brackets [].
# Good for function calls with lots of arguments
sum([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
     6, 7, 8, 9, 0])
print("This is my super looooooooooooooooooooooooooooooong string "
     "and it ends here!")

# Implicit string concatenation: notice that string literals are automatically appended to each other without '+' operator
print("asdf" "asdf" == 'asdfasdf')
# Hence, parenthesis and long strings work well together.
my_long_string = ('This is my super long string that never ends because I do not want to stop '
                  "typing for some reason until I'm out of breath!")
my_long_string_2 = ('I like to count to big numbers. I start at {} '
                    'and finally end up at {}'
                    .format(1, int(1e10)))
print(my_long_string)
print(my_long_string_2)

This is my super looooooooooooooooooooooooooooooong string and it ends here!
True
This is my super long string that never ends because I do not want to stop typing for some reason until I'm out of breath!
I like to count to big numbers. I start at 1 and finally end up at 10000000000


In [ ]:
# PEP8 is a nice style guide for readability. I try my best, but even I don't get it right every time.
# http://pymbook.readthedocs.io/en/latest/pep8.html
# If you are very fancy, you can have Python automatically format your code to conform to pep8 if you type this in the Terminal
autopep8 your_python_script_here.py
# This will print to Terminal the correctly formatted script but doesn't save it. 
# If you want to save the results back into your script, you can use the argument --in-place
autopep8 --in-place your_python_script_here.py

In [3]:
# I usually use triple hash sign ### when I need to make an important comment. Usually to mention something is hard coded.
# This is not a Python PEP8 style. It's just a personal preference to make note that this is not a regular comment.
PI = 3.14159265 ### this is hard coded

As a personal preference, I prefer to write pure functions with no side effects where if you put in the same input, you always get the same output. When practical, I try to avoid functions that mutate whatever is inputted. Pure functions are easier to debug. Of course, sometimes it's just much easier or runs faster with mutation. Then I add a triple hash sign to say that this function mutates the underlying object.

In [1]:
class Issue():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
bbw = Issue(title="Bloomberg", price=5.99, pages=112)

TODOS:  
object() as sentinel value  
patterns?  singleton, module as a singleton  
new features: ordered dictionary, walrus operator, type annotations/static typing  